In [1]:
import open3d as o3d
import numpy as np
import copy
import time
import os 
import sys
import cv2
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Sources:
#front_view_translated_path = "CenteredPointClouds/oil_pan_front_pc_10000.ply"
#side_view_translated_path = "CenteredPointClouds/oil_pan_sv_pc_10000.ply" 
# Target:
#full_model_translated_path = "CenteredPointClouds/oil_pan_full_pc_10000.ply" 

# Sources
oil_pan_front_pc_path = "PointClouds/oil_pan_front_pc_10000.ply"
oil_pan_sv_pc_path = "PointClouds/oil_pan_sv_pc_10000.ply"
# Target
oil_pan_full_pc_path = "PointClouds/oil_pan_full_pc_10000.ply"

# Giving them their names:
source_path=oil_pan_sv_pc_path
target_path=oil_pan_full_pc_path

Helper function for visualization

In [3]:
def draw_registration_result(source, target, transformation):
  source_temp = copy.deepcopy(source)
  target_temp = copy.deepcopy(target)
  source_temp.paint_uniform_color([1, 0.706, 0])
  target_temp.paint_uniform_color([0, 0.651, 0.929])
  source_temp.transform(transformation)
  o3d.visualization.draw_geometries([source_temp, target_temp])

Global Registration

In [4]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size of %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with  search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down, 
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )
    return pcd_down, pcd_fpfh

In [5]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb the initial pose.")
    source = o3d.io.read_point_cloud(source_path)
    target = o3d.io.read_point_cloud(target_path)
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], 
                             [1.0, 0.0, 0.0, 0.0], 
                             [0.0, 1.0, 0.0, 0.0], 
                             [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    o3d.io.write_point_cloud("transformed_source_front_side.ply", source)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [6]:
voxel_size = 1 # means 5cm for the original dataset... gotta check on mine


source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size=voxel_size)

:: Load two point clouds and disturb the initial pose.
:: Downsample with a voxel size of 1.000.
:: Estimate normal with  search radius 2.000.
:: Compute FPFH feature with search radius 5.000.
:: Downsample with a voxel size of 1.000.
:: Estimate normal with  search radius 2.000.
:: Compute FPFH feature with search radius 5.000.


RANSAC

In [7]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds. ")
    print("   Since the downsampling voxel sixe is %.3f, " % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, 
        distance_threshold, 
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3, 
        [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9
            ),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold
            )
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )
    return result

In [8]:
result_ransac = execute_global_registration(source_down, target_down, 
                                            source_fpfh, target_fpfh, 
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds. 
   Since the downsampling voxel sixe is 1.000, 
   we use a liberal distance threshold 1.500.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.426146e-01, and correspondence_set size of 1037
Access transformation to get result.


ICP

In [9]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation, 
        o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    return result

In [10]:
def compute_normals(pcd, radius):
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30)
    )



In [11]:
radius_normal = voxel_size * 2
compute_normals(source, radius_normal)
compute_normals(target, radius_normal)

In [12]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh, voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.400.
RegistrationResult with fitness=9.974000e-01, inlier_rmse=2.046595e-01, and correspondence_set size of 9974
Access transformation to get result.
